# Bayesian Linear Regression with Pyro

The goal of this notebook is to show how to create a bayesian linear regression model for a simple dataset with only one input dimension and one output dimension. All examples I could find immediately go into much more detail on more complex datasets. While for me it's important to understand this simple examples first.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyro
import pyro.optim
import torch

from pyro.contrib.autoguide import AutoDiagonalNormal

%matplotlib inline

## Generate toy dataset

We generate a toy linear dataset with a bit of noise.

In [ ]:
size = 100

X = np.linspace(0, 1, size)
Y_true = 2 * X + 1
Y = Y_true + np.random.normal(scale=.2, size=size)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.plot(X, Y, 'x', label='Sample data')
ax.plot(X, Y_true, label='Ground truth', lw=2)
ax.legend()

fig.show()

## Create a regression model

We create a regression model with pytorch and train the regression model to fit the sampled data.

In [ ]:
class RegressionModel(torch.nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.linear = torch.nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)

In [ ]:
# If available use CUDA
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
regression_model = RegressionModel().to(device)

print('Random initialized regression model parameters:')
for name, param in regression_model.named_parameters():
    print(name, param.data.numpy())

In [ ]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(regression_model.parameters(), lr=0.01)
num_epochs = 1000

x_tensor = torch.tensor(X, dtype=torch.float).unsqueeze(1).to(device)
y_tensor = torch.tensor(Y, dtype=torch.float).unsqueeze(1).to(device)

for epoch in range(num_epochs):
    y_prediction = regression_model(x_tensor)
    cur_loss = loss(y_prediction, y_tensor)
    optimizer.zero_grad()
    cur_loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0 or epoch == num_epochs - 1:
        print(f'Epoch {epoch}, loss {cur_loss:.4f}')
        
print('\nLearned parameters:')
for name, param in regression_model.named_parameters():
    print(name, param.data.numpy())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.plot(X, Y, 'x', label='Sampled data')
ax.plot(X, Y_true, label='Ground truth', lw=2)
ax.plot(X, regression_model(x_tensor).detach().cpu().numpy().squeeze(), label='Predicted values')
ax.legend()

fig.show()

## Create a Bayesian linear regression model

In [ ]:
def bayesian_model(x_data, y_data):
    w_prior = pyro.distributions.Normal(
        torch.tensor([[0.]]).to(device),
        torch.tensor([[1.]]).to(device)).to_event(1)
    b_prior = pyro.distributions.Normal(
        torch.tensor([[0.]]).to(device),
        torch.tensor([[1.]]).to(device)).to_event(1)
    
    priors = {
        'linear.weights': w_prior,
        'linear.bias': b_prior
    }
    
    scale = pyro.sample('sigma', pyro.distributions.Uniform(0., 5.))
    lifted_module = pyro.random_module('module', regression_model, priors)
    lifted_regression_model = lifted_module()
    
    with pyro.plate('map', len(x_data)):
        prediction_mean = lifted_regression_model(x_data).squeeze(1)
        if y_data is not None:
            pyro.sample(
                'obs',
                pyro.distributions.Normal(prediction_mean, scale),
                obs=y_data.squeeze(1))
        else:
            pyro.sample('obs', pyro.distributions.Normal(prediction_mean, scale), obs=None)
        return prediction_mean

In [ ]:
guide = AutoDiagonalNormal(bayesian_model)

In [ ]:
optimizer = pyro.optim.Adam({'lr': 0.01})
svi = pyro.infer.SVI(
    bayesian_model,
    guide,
    optimizer,
    loss=pyro.infer.Trace_ELBO(),
    num_samples=1000)

In [ ]:
pyro.clear_param_store()
for epoch in range(3000):
    cur_loss = svi.step(x_tensor, y_tensor)
    if epoch % 100 == 0 or epoch == num_epochs - 1:
        print(f'Epoch {epoch}, loss {cur_loss / len(x_tensor)}')
        
print('\nLearned parameters:')
for name, param in pyro.get_param_store().items():
    print(name, param.data.numpy())

## Model evaluation

In [ ]:
def get_marginals(traces, sites):
    samples_and_weights = pyro.infer.EmpiricalMarginal(traces, sites)._get_samples_and_weights()[0]
    return samples_and_weights.detach().cpu().numpy()

In [ ]:
def evaluation_bayesian_model(x_data, y_data):
    pyro.sample('prediction', pyro.distributions.Delta(bayesian_model(x_data, y_data)))
    
posterior = svi.run(x_tensor, y_tensor)

In [ ]:
trace_prediction = pyro.infer.TracePredictive(evaluation_bayesian_model, posterior, num_samples=1000)
post_prediction = trace_prediction.run(x_tensor, None)

In [ ]:
post_marginals = get_marginals(post_prediction, ['prediction', 'obs'])
predictions = post_marginals[:, 0, :]
observations = post_marginals[:, 1, :]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.plot(X, Y, 'x', label='Sampled data')
ax.plot(X, Y_true, label='Ground truth', lw=2)

q05 = np.quantile(predictions, .001, axis=0)
q95 = np.quantile(predictions, .999, axis=0)
ax.plot(X, predictions.mean(axis=0), label='Bayesian predictions', lw=2)
ax.fill_between(X, q05, q95, alpha=0.1)
ax.legend()

fig.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.plot(X, Y, 'x', label='Sampled data')
ax.plot(X, Y_true, label='Ground truth', lw=2)

q05 = np.quantile(observations, .05, axis=0)
q95 = np.quantile(observations, .95, axis=0)
ax.plot(X, observations.mean(axis=0), label='Bayesian predictions', lw=2)
ax.fill_between(X, q05, q95, alpha=0.1)
ax.legend()

fig.show()